In [1]:
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

2021-11-15 19:47:21.174811: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-15 19:47:21.174869: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Jupyter notebook to build, train, and deply CNN models for TF Lite

In [2]:
filename = "tiny_vgg_q" # Set the name for the model output

early_stop_training = True # Early stop training

#___________________________
training_batch_size = 20
training_epochs = 100
training_patience = 10

#___________________________
quantize_aware_training = True
quantize_training_epochs = 100
quantize_training_patience = 10

target_exponent = 3 # Target exponent bit size for custom floating-point 
target_mantissa = 0 # Target mantissa bit size for custom floating-point

max_degradation = 0.01

In [3]:
google_colab = False # Set True when using on google colab
if google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive')

CNN ARCHITECTURE:

In [4]:
# define cnn model
def ConvPool_CNN_C():
    model = Sequential()

    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', input_shape=(32, 32, 3)))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (1, 1), activation='relu'))
    model.add(Conv2D(10, (1, 1)))
    model.add(GlobalAveragePooling2D())
    model.add(Activation(activation='softmax'))

    # compile model
    #opt = SGD(learning_rate=0.001, momentum=0.9)
    opt = "adam"
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# define cnn model
def shallow_CNN():
	model = Sequential()
	
	model.add(Conv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [6]:
# define cnn model
def shallow_dropout_CNN():
	model = Sequential()
	
	model.add(Conv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [7]:
# define cnn model
def shallow_SeparableCNN():
	model = Sequential()
	
	model.add(SeparableConv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(SeparableConv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(SeparableConv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(120, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dropout(0.5))
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [8]:
# define cnn model
def define_model_CNN():
	return shallow_dropout_CNN()

Quantized aware training method.

In [9]:
import struct

def bin2float(b):
    ''' Convert binary string to a float.

    Attributes:
        :b: Binary string to transform.
    '''
    h = int(b, 2).to_bytes(8, byteorder="big")
    return struct.unpack('>d', h)[0]


def float2bin(f):
    ''' Convert float to 64-bit binary string.

    Attributes:
        :f: Float number to transform.
    '''
    [d] = struct.unpack(">Q", struct.pack(">d", f))
    return f'{d:064b}'

In [10]:
def quantize_float (float_number, exponent_bits, mantissa_bits):
    exponent_sign = 1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    if mantissa_bits < 0:
      mantissa_bits = 0
    if exponent_bits < 0:
      exponent_bits = 0
      
    coefficient = float2bin(float_number)
    sign = int (coefficient[:1], 2)
    if 0 < mantissa_bits:
        custom_mantissa = int (coefficient[12 : 12 + mantissa_bits], 2)
    else:
        custom_mantissa = 0
    residual_mantissa = int (coefficient[12 + mantissa_bits:], 2)
    exponent = int (coefficient[1:12], 2) - 1023

    exponent_full_range = pow(2, exponent_bits - exponent_sign) - 1
    if exponent < - exponent_full_range:
        quantized_value = 0
    elif exponent > exponent_full_range:
        quantized_value = pow(-1, sign) * (1 + (1 - pow(2, - mantissa_bits))) * pow(2, exponent_full_range)
    else:
        if (pow (2, (52 - (mantissa_bits + 1))) - 1) < residual_mantissa:
            custom_mantissa += 1
            if (pow (2, mantissa_bits) - 1) < custom_mantissa:
                custom_mantissa = 0
                exponent += 1
    
        quantized_value = pow(-1, sign) * (1 + custom_mantissa * pow(2, - mantissa_bits)) * pow(2, exponent)
    return quantized_value

def quantize_model(model, exponent_bits, mantissa_bits):
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
      layer_weights = layer.get_weights()
      for index in range(len(layer_weights)):
        matrix = layer_weights[index]
        for weight_index, weight in np.ndenumerate(matrix):
          matrix[weight_index] = quantize_float(weight, target_exponent, target_mantissa)
        layer_weights[index] = matrix
      layer.set_weights(layer_weights)

Early stop callback.

In [11]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto', restore_best_weights=True)

In [12]:
mantissa_bits = target_mantissa
exponent_bits = target_exponent
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = None):
    quantize_model(self.model, exponent_bits, mantissa_bits)
  def on_train_end(self, logs = None):
    quantize_model(self.model, exponent_bits, mantissa_bits)
  def on_batch_end(self, epoch, logs = None):
    quantize_model(self.model, exponent_bits, mantissa_bits)


In [13]:
if not os.path.exists(filename):
  os.mkdir(filename)

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	f = pyplot.figure()
	f.set_figwidth(10)
	f.set_figheight(20)
	
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	pyplot.savefig(filename + '/' + filename + '_plot.png')
	pyplot.close()

def print_model(model):
	for layer in model.layers:
		if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
			layer_weights = layer.get_weights()
			print (layer_weights)

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model_CNN()
	# fit model
	history = model.fit(trainX, trainY, epochs=20, batch_size=64, validation_data=(testX, testY), verbose=1, callbacks=[MyCallback()])
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
	# Save the entire model to a HDF5 file.
	model.save(filename + "/" + filename + '.h5')

In [14]:
# entry point, run the test harness
# load dataset
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

# define model
model = define_model_CNN()
# fit model

2021-11-15 19:48:05.562629: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-15 19:48:05.626319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 19:48:05.627972: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 860M computeCapability: 5.0
coreClock: 1.0195GHz coreCount: 5 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 74.65GiB/s
2021-11-15 19:48:05.629872: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-15 19:48:05.632369: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libc

In [15]:
if not early_stop_training:
    history = model.fit(trainX, trainY, epochs=training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1)
else:
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=training_patience, verbose=1, mode='auto', restore_best_weights=True)
    history = model.fit(trainX, trainY, epochs=training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1, callbacks=[monitor])

# evaluate model
_, pre_acc = model.evaluate(testX, testY, verbose=0)
print('Pre-quantization training > %.3f' % (pre_acc * 100.0))

2021-11-15 19:48:07.400983: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/100
2500/2500 [==============================] - ETA: 0s - loss: 1.9885 - accuracy: 0.2394

2021-11-15 19:51:18.265289: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 122880000 exceeds 10% of free system memory.


2500/2500 [==============================] - 197s 79ms/step - loss: 1.9885 - accuracy: 0.2394 - val_loss: 1.5909 - val_accuracy: 0.4319
Epoch 2/100
2500/2500 [==============================] - 198s 79ms/step - loss: 1.5507 - accuracy: 0.4299 - val_loss: 1.2742 - val_accuracy: 0.5429
Epoch 3/100
2500/2500 [==============================] - 199s 80ms/step - loss: 1.3899 - accuracy: 0.5005 - val_loss: 1.1332 - val_accuracy: 0.5983
Epoch 4/100
2500/2500 [==============================] - 187s 75ms/step - loss: 1.2879 - accuracy: 0.5430 - val_loss: 1.0461 - val_accuracy: 0.6426
Epoch 5/100
2500/2500 [==============================] - 181s 72ms/step - loss: 1.2087 - accuracy: 0.5730 - val_loss: 0.9902 - val_accuracy: 0.6555
Epoch 6/100
2500/2500 [==============================] - 181s 72ms/step - loss: 1.1529 - accuracy: 0.5938 - val_loss: 0.9445 - val_accuracy: 0.6738
Epoch 7/100
2500/2500 [==============================] - 177s 71ms/step - loss: 1.1132 - accuracy: 0.6093 - val_loss: 1.0100

2021-11-15 22:45:22.522799: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 122880000 exceeds 10% of free system memory.


Pre-quantization training > 76.850


In [16]:
# learning curves
summarize_diagnostics(history)

# Save the entire model to a HDF5 file.
model.save(filename + "/" + filename + '.h5')

In [17]:
print ("_______ Post-training quantization _______")
model = load_model(filename + "/" + filename + '.h5')
_, pre_acc = model.evaluate(testX, testY, verbose=0)
print('Original accuracy > %.3f' % (pre_acc * 100.0))

quantize_model(model, exponent_bits, mantissa_bits)
# evaluate model
_, current_acc = model.evaluate(testX, testY, verbose=0)
print_model (model)
print('Post-training quantization > %.3f' % (current_acc * 100.0))

print("exponent = " + str(exponent_bits) + ", mantissa = " + str(mantissa_bits))

if quantize_aware_training:
    print ("_______ Quantize aware training _______")
    quantize_loop = True
    while quantize_loop:
        print ("Starting...")
        model = load_model(filename + "/" + filename + '.h5')
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=quantize_training_patience, verbose=1, mode='auto', restore_best_weights=True)
        history = model.fit(trainX, trainY, epochs=quantize_training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1, callbacks=[MyCallback(), monitor])
        # learning curves
        summarize_diagnostics(history)
        # evaluate model
        _, test_acc = model.evaluate(testX, testY, verbose=0)
        print('Quantize aware training > %.3f' % (current_acc * 100.0))
        
        if current_acc < test_acc:
            current_acc = test_acc
            model.save(filename + "/" + filename + '.h5')
            print_model (model)
            print('Save model > %.3f' % (current_acc * 100.0))
            quantize_loop = current_acc < pre_acc - max_degradation
        
    print('Ending quantized aware training > %.3f' % (current_acc * 100.0))

_______ Post-training quantization _______


2021-11-15 22:45:37.170182: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 122880000 exceeds 10% of free system memory.


Original accuracy > 76.850


2021-11-15 22:45:47.170939: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 122880000 exceeds 10% of free system memory.


[array([[[[-0.25 , -0.25 , -0.125, ..., -0.5  , -0.25 ,  0.25 ],
         [ 0.   ,  0.   , -0.5  , ..., -0.5  ,  0.25 , -0.125],
         [ 0.25 ,  0.5  ,  0.   , ..., -0.25 ,  0.5  , -0.5  ]],

        [[ 0.5  ,  0.   , -0.25 , ...,  0.5  , -0.25 ,  0.25 ],
         [-0.5  , -0.125, -0.25 , ...,  0.125, -0.25 ,  0.   ],
         [-0.25 ,  0.   , -0.25 , ...,  0.   , -0.5  ,  0.   ]],

        [[ 0.25 , -0.25 ,  1.   , ...,  0.25 ,  0.5  ,  0.125],
         [-1.   , -0.25 ,  0.5  , ..., -0.5  ,  0.   ,  0.   ],
         [ 0.5  ,  0.25 ,  0.   , ...,  0.5  ,  0.125,  0.   ]]],


       [[[-0.25 ,  0.   , -0.125, ..., -0.25 ,  0.25 ,  0.5  ],
         [-0.25 , -0.25 ,  0.   , ..., -0.5  ,  0.   ,  0.   ],
         [ 0.5  ,  0.   ,  0.   , ...,  0.25 ,  0.5  , -1.   ]],

        [[ 0.5  , -0.25 ,  0.125, ...,  0.   , -0.125,  0.   ],
         [-0.25 ,  0.5  , -0.125, ..., -0.5  , -0.5  ,  0.   ],
         [ 0.125,  0.25 ,  0.   , ..., -0.5  , -0.5  , -0.5  ]],

        [[ 0.25 , -0.5  , -

KeyboardInterrupt: 

In [ ]:
# Save the entire model to a HDF5 file.
model.save(filename + "/" + filename + '.h5')

_______ LOAD CNN MODEL FOR EVALUATION AND CONVERSION TO TF LITE FLOATING-POINT AND FIXED-POINT _______

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

img_path = "dog.jpg"
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x.astype('float32')
x = x / 255.0

model = load_model(filename + "/binary.h5")

output_data = model.predict(x)

print('TensorFlow Predicted:', output_data)

print("Done!")

TensorFlow Predicted: [[1.1118077e-02 4.2626081e-04 6.1177625e-03 3.2010840e-03 8.2191473e-01
  3.4573304e-03 9.0579040e-02 1.0590014e-03 4.7753200e-02 1.4373433e-02]]
Done!


In [ ]:
def print_model(model):
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
      layer_weights = layer.get_weights()
      print (layer_weights)

In [ ]:
print_model (model)
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))

[array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  1.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0., -1.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 1.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0., 

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils

if not os.path.exists(filename):
  print("Folder " + filename + " does not exist.")
  exit()

# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(filename + "/" + filename + '.h5')

model.summary()

tf.keras.utils.plot_model(model, filename + "/" + filename + ".png", show_shapes=True, show_layer_names=True, expand_nested=True)

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255 , test_images / 255

train_images = tf.cast(train_images, tf.float32)
cifar_ds = tf.data.Dataset.from_tensor_slices((train_images)).batch(1)
def representative_dataset():
  for input_value in cifar_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset = representative_dataset
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32  # or tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(filename + "/" + filename + "_f32" + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 40)        1120      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 40)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 40)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 40)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 60)        21660     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 60)        240       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 60)          0

KeyboardInterrupt: 

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32  # or tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(filename + "/" + filename + "_i8" + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

INFO:tensorflow:Assets written to: /tmp/tmpaw40s58t/assets


INFO:tensorflow:Assets written to: /tmp/tmpaw40s58t/assets
2021-11-12 23:11:07.428143: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 23:11:07.428853: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-11-12 23:11:07.428986: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-12 23:11:07.429833: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 23:11:07.430593: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 860M computeCapability: 5.0
coreClock: 1.0195GHz coreCount: 5 deviceMemorySize: 1.96GiB devi

Done!


In [ ]:

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=filename + "/" + filename + "_i8" + '.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:

interpreter.set_tensor(input_details[0]['index'], x)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print('TensorFlow Lite Predicted:', output_data)

TensorFlow Lite Predicted: [[0.         0.         0.05078125 0.08203125 0.17578125 0.20703125
  0.37890625 0.09765625 0.         0.00390625]]
